## Setup — Fluid–Structure Interaction (FSI)

We solve a **steady, incompressible Navier–Stokes** flow to obtain the fluid pressure field, then apply that pressure as a traction load on a **linear elastic** solid (one-way/weak coupling). The coupled governing equations are:

### Governing equations

$$
\rho\,(\mathbf{u}\!\cdot\!\nabla)\mathbf{u}
-\nabla\!\cdot\!\Bigl[\mu\bigl(\nabla\mathbf{u}+(\nabla\mathbf{u})^{\mathsf T}\bigr)\Bigr]
+\nabla p
+\alpha\,\mathbf{u}
= \mathbf{0}
\qquad \text{in } \Omega,
$$

$$
\nabla\!\cdot\!\mathbf{u}=0
\qquad \text{in } \Omega.
$$

**Structure (in $ \Omega_s $):**
$$
\begin{aligned}
\nabla\!\cdot\!\boldsymbol{\sigma}^s + \mathbf{f} &= \mathbf{0}
&&\text{in } \Omega_s,\\[4pt]
\boldsymbol{\sigma}^s &= \mathbf{C} : \boldsymbol{\epsilon}^s,\\[4pt]
\boldsymbol{\epsilon}^s &= \tfrac12\!\bigl(\nabla \mathbf{d}
+ (\nabla \mathbf{d})^{\mathsf T}\bigr).
\end{aligned}
$$

**Fluid–structure interface (pressure coupling on $ \Gamma_{fs} $):**
$$
\boldsymbol{\sigma}^s \!\cdot\! \mathbf{n} \;=\; -\,p\,\mathbf{n}
\qquad \text{on } \Gamma_{fs}.
$$

### Symbols

- $ \mathbf{d} $: structural displacement  
- $ \boldsymbol{\epsilon}^s $: small (linear) strain tensor  
- $ \boldsymbol{\sigma}^s $: Cauchy stress in the solid  
- $ \mathbf{C} $: fourth-order elasticity tensor (e.g., isotropic Hooke’s law)  
- $ \mathbf{f} $: body force density in the solid  
- $ \mathbf{n} $: unit outward normal on $ \partial\Omega_s $  
- $ p $: fluid pressure evaluated on $ \Gamma_{fs} $ from the flow solve

### Boundary conditions

**Fluid (in $ \Omega_f $):**
$$
\begin{aligned}
\mathbf{u} &= \mathbf{u}_{\text{in}}
&&\text{on } \Gamma_{\text{in}}\quad\text{(prescribed inflow)},\\[4pt]
-p\,\mathbf{n} + \mu\bigl(\nabla\mathbf{u}+(\nabla\mathbf{u})^{\mathsf T}\bigr)\mathbf{n} &= \mathbf{0}
&&\text{on } \Gamma_{\text{out}}\quad\text{(zero-normal-stress outflow)},\\[4pt]
\mathbf{u} &= \mathbf{0}
&&\text{on } \Gamma_{\text{wall}}\quad\text{(no-slip walls)}.
\end{aligned}
$$

**Structure (in $ \Omega_s $):**
$$
\begin{aligned}
\mathbf{d} &= \mathbf{0}
&&\text{on } \Gamma_{\text{fix}} \quad \text{(clamped/essential BC)},\\[4pt]
\boldsymbol{\sigma}^s\!\cdot\!\mathbf{n} &= -\,p\,\mathbf{n}
&&\text{on } \Gamma_{fs} \quad \text{(pressure from fluid)},\\[4pt]
\boldsymbol{\sigma}^s\!\cdot\!\mathbf{n} &= \bar{\mathbf{t}}
&&\text{on } \Gamma_t \quad \text{(optional, additional tractions)}.
\end{aligned}
$$

> *One-way coupling:* the structural deformation does **not** feed back into the flow; only the fluid pressure acts on the solid.

---

### Test case: FSI benchmark — pressure-loaded flexible insert

<img src="../../figures/fsi_solve.png" alt="FSI Channel" style="width: 500px;"/>

A steady laminar flow is solved in a 2-D channel. The resulting pressure along the
fluid–solid interface $ \Gamma_{fs} $ is transferred as a normal traction to a compliant
solid insert (e.g., a thin plate). A **mask** $ \Psi $ is used to activate the pressure
coupling in solid regions ($\Psi=1$ in solid, $\Psi=0$ in fluid). The solid is clamped at its base
($ \Gamma_{\text{fix}} $) and deforms under the applied pressure.

- **Flow field:** steady incompressible Navier–Stokes with optional Brinkman penalization (for TO); pressure $p$ is sampled on $ \Gamma_{fs} $.  
- **Structural model:** small-strain linear elasticity with isotropic Hooke’s law.  
- **Discretization:** bilinear quadrilateral elements ($Q_1$) for both fluid (Q1/Q1) and solid displacement; the same grid enables node-wise pressure transfer.  
- **Solution strategy:** solve flow $\Rightarrow$ assemble interface tractions (using the mask $ \Psi $) $\Rightarrow$ solve linear elasticity (damped Newton optional for contact/nonlinearities).


In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
import shapely
import shapely.geometry as shap_geom
import matplotlib.pyplot as plt

import toflux.src.utils as _utils
import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.bc as _bc

import toflux.src.fe_struct as _fea_struct
import toflux.src.fe_fluid as _fea_fluid
import toflux.experiments.fsi.pressure_coupling as _fsi
import toflux.src.solver as _solv
import toflux.src.viz as _viz


_Ext = _utils.Extent
_FluidField = _fea_fluid.FluidField
_Direc = _utils.Direction

jax.config.update("jax_enable_x64", True)

## Define Geometry and Mesh

### Mesh

Below we build the **fluid–structure interaction (FSI)** channel benchmark.

#### Geometry
The channel geometry (JSON) is passed to **both** meshers (fluid and structural).
The fluid subdomain $ \Omega_f $ and the solid subdomain $ \Omega_s $ share a **conforming interface** $ \Gamma_{fs} $, so nodes coincide for direct transfer of the interface traction $-p\,\mathbf{n}$.

#### Discretisation
* The domain is partitioned into **bilinear quadrilateral (Q1) elements** with **identical element counts** for the fluid and structural solves.  
* **Fluid:** equal-order **Q1/Q1** formulation for velocity–pressure on the shared grid (typical DOFs per node: $(p, u_x, u_y)$), with the usual stabilization as needed.  
* **Structure:** **Q1** displacement elements on the same grid (typical DOFs per node: $(d_x, d_y)$).  
* The conforming mesh enforces strong coupling on $ \Gamma_{fs} $: the fluid pressure $p$ is applied to the solid as a nodal/edge traction $-p\,\mathbf{n}$.
  (A mask $ \Psi $ is used, it activates coupling in solid regions: $ \Psi=1 $ in $ \Omega_s $, $ \Psi=0 $ in $ \Omega_f $.)

#### Numerical integration
Element contributions are evaluated with **$2\times2$** Gauss quadrature:
* **Fluid ($\Omega_f$):** convection, viscous, and optional Brinkman/stabilization terms.  
* **Solid ($\Omega_s$):** linear elastic stiffness and body-force terms.  
* **Interface ($\Gamma_{fs}$):** tractions assembled on the shared edges (2D) / faces (3D); no mortar or additional interface quadrature is required on a conforming mesh.

This mesh serves as the baseline for the finite-element **FSI** solve (steady flow $\rightarrow$ pressure coupling $\rightarrow$ linear elasticity) and for subsequent **density-based topology-optimisation** runs with compliance or other objectives.


In [ ]:
geom = _geom.BrepGeometry("toflux/brep/channel_flow_wall.json")

fluid_mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=150,
  nely_desired=75,
  dofs_per_node=3,
  gauss_order=2,
)

struct_mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=150,
  nely_desired=75,
  dofs_per_node=2,
  gauss_order=2,
)

_viz.plot_brep(geom)

## Material

For this **FSI** setup we specify **structural (elasticity)** and **fluid** properties.
`fluid_mat = _mat.FluidMaterial(ρ, μ)`.

### Structure (linear elasticity)

| Symbol | Meaning | Value |
|---|---|---|
| $E$ | Young’s modulus | $1.0\times 10^{5}$ |
| $\nu$ | Poisson’s ratio | $0.30$ |

- **Deformation model:** small strain ($\boldsymbol{\epsilon}^s=\tfrac12(\nabla\mathbf d+(\nabla\mathbf d)^{\mathsf T})$).  
- **Constitutive law:** isotropic Hooke’s law, $\boldsymbol{\sigma}^s=\mathbf C:\boldsymbol{\epsilon}^s$.  


### Fluid

| Symbol | Meaning | Value |
|---|---|---|
| $\rho$ | Mass density | $1.0$ |
| $\mu$ | Dynamic viscosity | $1.0$ |



In [ ]:
struct_mat = _mat.StructuralMaterial(
  youngs_modulus=1.0e5,
  poissons_ratio=0.30,
)

deformation_model = _fea_struct.DeformationModel.SMALL

fluid_mat = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=1.0,
)

## Structural boundary condition

The structural model applies a **clamped (Dirichlet)** constraint on the **bottom face** of the solid.

In [ ]:
face_tol = fluid_mesh.elem_size[0] * 0.5

btm_faces = _bc.identify_faces(struct_mesh, edges=[geom.edges[0]], tol=face_tol)
n = len(btm_faces)
xv = (_Direc.X, jnp.zeros(n))
yv = (_Direc.Y, jnp.zeros(n))
btm_face_val = [xv, yv]

fixed_bc = _bc.DirichletBC(elem_faces=btm_faces, values=btm_face_val, name="fixed")
struct_bc_list = [fixed_bc]
struct_bc = _bc.process_boundary_conditions(struct_bc_list, struct_mesh)

_viz.plot_bc(struct_bc_list, fluid_mesh)

## Fluid Boundray Conditions



| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet (left vertical edge)** | Dirichlet | $(u = U_\text{c})$, $(v = 0)$ |
| **Outlet (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a channel Reynolds number of 1:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 1,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{channel height}.
$$


In [ ]:
reynolds_num = 1.0
char_velocity = (
  reynolds_num * fluid_mat.kinematic_viscosity / (fluid_mesh.bounding_box.ly)
)

# inlet condition
inlet_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[3]], tol=face_tol)
n = len(inlet_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_FluidField.U_VEL, face_node_vals)
v_vel = (_FluidField.V_VEL, jnp.zeros_like(face_node_vals))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[1]], tol=face_tol)
n = len(outlet_faces)
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
pres = (_FluidField.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# wall condition
wall_faces = _bc.identify_faces(
  fluid_mesh, edges=[geom.edges[0], geom.edges[2]], tol=face_tol
)
n = len(wall_faces)
u_vel = (_FluidField.U_VEL, jnp.zeros(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
wall_face_val = [u_vel, v_vel]

inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="inlet")
outlet_bc = _bc.DirichletBC(
  elem_faces=outlet_faces, values=outlet_face_val, name="outlet"
)
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="wall")

fluid_bc_list = [inlet_bc, outlet_bc, wall_bc]
fluid_bc = _bc.process_boundary_conditions(fluid_bc_list, fluid_mesh)

_viz.plot_bc(fluid_bc_list, fluid_mesh)

# Solver

The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'. The structural equations are linear.  By default, all our FEA solvers are programmed as a subclass of nonlinear problems. Hence, the solver expects the settings for both the linear and nonlinear solvers.
Even when the physics is linear, the solver still performs a Newton–Raphson step; for a linear model this step converges in one iteration. 


In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.SCIPY_SPARSE,
    "rtol": 1.0e-3,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-4},
}

flow_solver = _fea_fluid.FluidSolver(
  mesh=fluid_mesh, bc=fluid_bc, material=fluid_mat, solver_settings=solver_settings
)

In [ ]:
press_vel = jnp.zeros((fluid_mesh.num_dofs,))
press_vel = press_vel.at[fluid_bc["fixed_dofs"]].set(fluid_bc["dirichlet_values"])

In [ ]:
brep_hh = "toflux/brep/hungry_horse.json"

with open(brep_hh, "r") as f:
  geometry = shapely.from_geojson(f.read())


density = np.zeros((struct_mesh.num_elems,))

for e in range(struct_mesh.num_elems):
  if geometry.contains(shap_geom.Point(struct_mesh.elem_centers[e])):
    density[e] = 1.0

_viz.plot_grid_mesh(struct_mesh, density)

# Solve
We provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
brinkman_penalty = density * 1e9
press_vel = _solv.modified_newton_raphson_solve(
  flow_solver, press_vel, brinkman_penalty
)

## Plot the fields

We visualize the velocity  and pressure fields here.

In [ ]:
press_vel_elem = press_vel[fluid_mesh.elem_dof_mat]
press_elem = np.mean(press_vel_elem[:, 0::3], axis=1)
u_vel_elem = np.mean(press_vel_elem[:, 1::3], axis=1)
v_vel_elem = np.mean(press_vel_elem[:, 2::3], axis=1)
vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)
_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=fluid_mesh, field=vel_elem_mag, ax=ax, colorbar=True)
ax.set_title("Velocity Magnitude")

_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=fluid_mesh, field=press_elem, ax=ax, colorbar=True)
ax.set_title("Pressure")
plt.show()

## Solve structural

Having solved for the pressure and velocity fields, we next compute the structural dispacements.


We first compute the structural force due to the pressure. For more details and implementation, we refer the readers to `pressure_coupling.py`.

Then, using the computed forces and other imposed structural boundary conditions, we compute the displacement fields.

In [ ]:
disp = jnp.zeros((struct_mesh.num_dofs,)) + 1.0e-6
disp = disp.at[struct_bc["fixed_dofs"]].set(struct_bc["dirichlet_values"])

In [ ]:
lam, mu = struct_mat.lame_parameters
density = density + 1.0e-2
lam, mu = lam * density, mu * density

coupling_force = _fsi.compute_pressure_force(
  press_vel[0 : fluid_mesh.num_dofs : 3], struct_mesh, density
)
struct_bc["elem_forces"] = coupling_force
struct_solver = _fea_struct.FEA(
  mesh=struct_mesh,
  material=struct_mat,
  deformation_model=deformation_model,
  bc=struct_bc,
  solver_settings=solver_settings,
)
disp = _solv.newton_raphson_solve(struct_solver, disp, lam, mu)

## Plot the fields

We visualize the displacement magnitude here.

In [ ]:
disp_elem = disp[struct_mesh.elem_dof_mat]
u_disp, v_disp = disp_elem[:, 0::2].mean(axis=1), disp_elem[:, 1::2].mean(axis=1)
disp_mag = np.sqrt(u_disp**2 + v_disp**2)

disp_mag = disp_mag * density

_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=struct_mesh, field=disp_mag, ax=ax, colorbar=True)
ax.set_title("Displacement Magnitude")
plt.show()